## Objective


### Objective:- 
- objective of this project is to use spark ML for topic modlling on tweets from twitter data using AWS (EC2, S3, Athena, Quick Sight), databrick, Spark(modelling and preprocesing)
###### Note:
    - It took much time to get coherence socre using spark LDA so used Gensim to comapre models from diffrent libraries.
    - I have also removed some output of the cell as output was too long and takes time to load.

### Purpose of Topic modelling:-
- Topic modeling on inflation tweets can help gain insights/understand people's sentiment and opinions regarding inflation. By analyzing the topics that comes from the tweets, businesses can understand the issues that are most important to population, the concerns they have about inflation, and the ways in which they are being impacted by it.

- For example, if a topic emerges around the rising cost of food, businesses in the food industry could use this information to make impactful strategies. If another topic emerges around job security, businesses could use this information to understand how citizen/population are feeling about their employment opportuities and can adjust their labor planning or marketing strateg.

## Workflow
- Mount the s3 bucket(weclouddata)
- Mount the Twitter project folder required for the project (Inflation)
- Data cleaning
- Feature Enginnering & EDA
- Data preparation
- Model training (Baseline Model / gridsearch)
- Model evaluation
- Topic Interpretation


#### Installing Libraries and Modules 

In [0]:
!pip install --upgrade pip
!pip install gensim==4.1.2
!pip install contractions
!pip install nltk
!pip install pyLDAvis

     |████████████████████████████████| 2.1 MB 6.6 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/c0/f4/278e305e02245937579a7952b8a3205116b4d2480a3c03fa11e599b773d6/regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
#spark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import udf, regexp_replace, split, col, array_except, explode
import contractions
import unicodedata
import matplotlib.pyplot as plt
from pyspark.ml.feature import NGram, VectorAssembler, StopWordsRemover, HashingTF, IDF, RegexTokenizer, StringIndexer, VectorAssembler,CountVectorizer,SQLTransformer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import LDA
from pyspark.ml.linalg import VectorUDT

# gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
nltk.download('stopwords')
nltk.download('wordnet')

import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


##Mounting S3 bucket

##### a. Function to mount to weclouddata public Twitter bucket

In [0]:
def mount_s3_bucket(access_key, secret_key, bucket_name, mount_folder):
    ACCESS_KEY_ID = access_key
    SECRET_ACCESS_KEY = secret_key
    ENCODED_SECRET_KEY = SECRET_ACCESS_KEY.replace("/", "%2F")

    print ("Mounting", bucket_name)

    try:
    # Unmount the data in case it was already mounted.
      dbutils.fs.unmount("/mnt/%s" % mount_folder)
    
    except:
    # If it fails to unmount it most likely wasn't mounted in the first place
      print ("Directory not unmounted: ", mount_folder)
    
    finally:
    # Lastly, mount our bucket.
      dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY_ID, ENCODED_SECRET_KEY, bucket_name), "/mnt/%s" % mount_folder)
    #dbutils.fs.mount("s3a://"+ ACCESS_KEY_ID + ":" + ENCODED_SECRET_KEY + "@" + bucket_name, mount_folder)
      print ("The bucket", bucket_name, "was mounted to", mount_folder, "\n")

##### b. Setting access key and secret access key

In [0]:
# Set AWS programmatic access credentials
ACCESS_KEY = ""
SECRET_ACCESS_KEY = ""

##### c. Mounting the twitter folder required for the project

In [ ]:
mount_s3_bucket(ACCESS_KEY, SECRET_ACCESS_KEY, "bucketname/foldername/", "twitter_projects")

In [ ]:
%fs ls /mnt/twitter_projects/

## Project on Inflation dataset

## 1. Reading Inflation Dataset (user-defined Schema)

In [0]:
### Project on Inflation dataset

spark = (SparkSession
        .builder
        .appName("twitter_projects")
        .getOrCreate()
        )

sc = spark.sparkContext

In [0]:
# Required for StructField, StringType, IntegerType, etc.

twitterSchema = StructType([
  StructField("id", StringType(), True),
  StructField("user_name", StringType(), True),
  StructField("screen_name", StringType(), True),
  StructField("tweet", StringType(), True),
  StructField("followers_count", IntegerType(), True),
  StructField("location", StringType(), True),
  StructField("geo", StringType(), True),
  StructField("created_at", StringType(), True)
])

In [0]:
df = (
    spark
    .read
    .option("header", "false")
    .option("delimiter", "\t")
    .option("encoding", "utf-8")
    .schema(twitterSchema)
    .csv("/mnt/twitter_projects/Inflation/*/*/*/*/*")
)

In [0]:
display(df.take(5))

id,user_name,screen_name,tweet,followers_count,location,geo,created_at
1602659952749076480,Marc Burr,marcburr,"RT @MorningBrew: If there's one thing the market loves, it's better than expected inflation data. Happy Tuesday everyone. https://t.co/jjGc…",337,"Billings, Montana",None,Tue Dec 13 13:41:23 +0000 2022
1602659953285951493,STOCK TRAIN,stocktrain2,10-year Treasury yield drops below 3.5% after inflation reading @CNBC https://t.co/qwJgRZngMw,34024,None,None,Tue Dec 13 13:41:23 +0000 2022
1602659953621204992,Caleb Kaplan,CalebKaplan,"RT @RepJasonSmith: The $2 trillion American Rescue Plan sparked the worst inflation in 40 years, forcing every family to pay $8,600 more th…",2050,"Georgia, USA",None,Tue Dec 13 13:41:23 +0000 2022
1602659955949043712,JD,JaedenJD,RT @BusinessInsider: Inflation cooled again in November to the slowest pace in a year https://t.co/CBUM5wQpWb,182,Singapore,None,Tue Dec 13 13:41:24 +0000 2022
1602659956016418816,Crutcial News of Crypto’s,CrusNewsCrypto,BREAKING: U.S. inflation slowed again last month in the latest sign that price increases are slowly cooling despite… https://t.co/GsyGAFPHof,51446,None,None,Tue Dec 13 13:41:24 +0000 2022


## 2. Data Profiling

In [0]:
# Counting number of rows in DF
row_count = df.count()

# Counting number of distinct values in each column
distinct_counts = df.agg(*[countDistinct(col_name).alias(col_name) for col_name in df.columns])

# Calculating the avg length of the 'tweet' column
avg_text_length = df.agg(avg(length('tweet'))).collect()[0][0]

print("Number of Rows: ", row_count)
print("Distinct Counts: ")
display(distinct_counts)
print("Average Text Length: ", avg_text_length)


Number of Rows:  889928
Distinct Counts: 


id,user_name,screen_name,tweet,followers_count,location,geo,created_at
889417,407277,451405,382506,41402,90290,22,425979


Average Text Length:  126.51989012187076


In [0]:
# Counting No. of nulls
null_counts = df.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Checking for rows with null values in 'tweet' column
null_tweet_rows = df.filter(col("tweet").isNull())

# Checking for rows with empty strings in the 'tweet' column
empty_tweet_rows = df.filter(trim(col("tweet")) == '')

# Counting No. of duplicates in DF
duplicates_count = df.count() - df.dropDuplicates().count()

# Printing all the results
print("Null Counts: ")
display(null_counts)
print("Null Tweet Rows: ")
display(null_tweet_rows)
print("Empty Tweet Rows: ")
display(empty_tweet_rows)
print("Duplicates Count: ", duplicates_count)

Null Counts: 


id,user_name,screen_name,tweet,followers_count,location,geo,created_at
76,89,72,215,1104,1106,1251,1251


Null Tweet Rows: 


id,user_name,screen_name,tweet,followers_count,location,geo,created_at
taboo and trauma kinks. 1996.,None,Tue Dec 13 14:01:44 +0000 2022,null,null,null,null,null
meet,None,Tue Dec 13 14:10:12 +0000 2022,null,null,null,null,null
null,None,Tue Dec 13 15:47:44 +0000 2022,null,null,null,null,null
null,None,Wed Dec 14 19:59:29 +0000 2022,null,null,null,null,null
any prns n/sfw mdni 21 yrs miko’s,None,Tue Dec 13 17:08:26 +0000 2022,null,null,null,null,null
null,None,Wed Dec 14 20:05:54 +0000 2022,null,null,null,null,null
null,None,Tue Dec 13 17:45:00 +0000 2022,null,null,null,null,null
meet,None,Thu Dec 15 19:32:03 +0000 2022,null,null,null,null,null
"Anchorage, AK, United States 99503",None,Tue Dec 13 16:22:28 +0000 2022,null,null,null,null,null
null,None,Tue Dec 13 16:36:32 +0000 2022,null,null,null,null,null


Empty Tweet Rows: 


id,user_name,screen_name,tweet,followers_count,location,geo,created_at


Duplicates Count:  405


## 3. Data Cleaning
- Dropping duplicates and null(Tweet values)
- Lower-Case Conversion
- Standardizing Accent Characters
- Removing URLs
- Removing Mentions and creating another column for Hashtags
- Expanding Contractions
- Removing special characters
- Multiple spaces to single space
- Trimming leading and trailing spaces

In [0]:
# Droppign duplicates and null(tweets values) from df
df1 = df.dropDuplicates().na.drop(subset=['tweet'])
# df1.count()

###### Cleaning tweets text for modelling

In [0]:
def standardize_accents(tweet):
    return unicodedata.normalize('NFKD', tweet).encode('ASCII', 'ignore').decode('utf-8')

udf_standardize_accents = udf(standardize_accents, StringType())

In [0]:
# Define a UDF to expand contractions
def udf_expand_contractions(tweet):
    expanded_words = [] 
    for word in tweet.split():
        expanded_words.append(contractions.fix(word)) 
    return ' '.join(expanded_words)

# Create a UDF from the udf_expand_contractions function
udf_expand_contractions = udf(udf_expand_contractions, StringType())

###### Cleaning location column for eda and athena queries

In [ ]:
df_clean = df1.withColumn('location', lower('location')) \
             .withColumn('location', when(col('location') == 'none', 'NA').otherwise(col('location'))) \
             .withColumn('location', regexp_replace('location', r"http\S+", "")) \
             .withColumn('location', regexp_replace('location', '@\\S+', '')) \
             .withColumn('location', regexp_replace('location', '#\\S+', '')) \
             .withColumn('location', regexp_replace('location', r"[^a-zA-Z]", " ")) \
             .withColumn('location', regexp_replace('location', r"\s+", " ")) \
             .withColumn('location', trim('location'))

display(df_clean)


### OUTPUT Example
# id	user_name	screen_name	tweet	followers_count	location	geo	created_at
# 1602660120676036608	Deduped	deduped_nytimes	RT @nytimes: Breaking News: U.S. inflation cooled more than expected in November, an encouraging sign for the economy. Consumer prices rose…	636	NA	None	Tue Dec 13 13:42:03 +0000 2022
# 1602660601620201472	Gregory Daco	GregDaco	🇺🇸Inflation coming down! #CPI +0.1% in November (<expected) 🟢Core CPI +0.2% 🍲Food +0.5% ⛽️Energy -1.6% (gas -2%… https://t.co/766DTk5tnv	15292	new york ny	None	Tue Dec 13 13:43:58 +0000 2022

In [ ]:
df_clean = df1.withColumn('tweet', F.lower('tweet')) \
             .withColumn('tweet', udf_standardize_accents('tweet')) \
             .withColumn('tweet', F.regexp_replace('tweet', r"http\S+", "")) \
             .withColumn('tweet', regexp_replace(col('tweet'), '@\\S+', '')) \
             .withColumn('tweet', regexp_replace(col('tweet'), '#\\S+', '')) \
             .withColumn('tweet', udf_expand_contractions('tweet')) \
             .withColumn('tweet', F.regexp_replace('tweet', r"[^a-zA-Z]", " ")) \
             .withColumn('tweet', F.regexp_replace('tweet', r"\s+", " ")) \
             .withColumn('tweet', F.trim('tweet'))

display(df_clean)

### OUTPUT Example
# id	user_name	screen_name	tweet	followers_count	location	geo	created_at
# 1602660120676036608	Deduped	deduped_nytimes	RT @nytimes: Breaking News: U.S. inflation cooled more than expected in November, an encouraging sign for the economy. Consumer prices rose…	636	NA	None	Tue Dec 13 13:42:03 +0000 2022
# 1602660601620201472	Gregory Daco	GregDaco	🇺🇸Inflation coming down! #CPI +0.1% in November (<expected) 🟢Core CPI +0.2% 🍲Food +0.5% ⛽️Energy -1.6% (gas -2%… https://t.co/766DTk5tnv	15292	new york ny	None	Tue Dec 13 13:43:58 +0000 2022

In [0]:
df_clean = df_clean.drop(col("id")).drop(col("user_name")).drop(col("followers_count"))

In [0]:
df_clean.cache()
df_clean.count()

#### Saving Clean df to our S3(AWS) Bucket 

In [0]:
mount_s3_bucket(ACCESS_KEY, SECRET_ACCESS_KEY, "b17disha/Inflation_project", "inflation")

Mounting b17disha/Inflation_project
/mnt/inflation has been unmounted.
The bucket b17disha/Inflation_project was mounted to inflation 



In [0]:
%fs ls /mnt/inflation/

path,name,size,modificationTime
dbfs:/mnt/inflation/Location.csv/,Location.csv/,0,0
dbfs:/mnt/inflation/Spark_LDA_predictions.parquet/,Spark_LDA_predictions.parquet/,0,0
dbfs:/mnt/inflation/Tweets_df.csv/,Tweets_df.csv/,0,0


In [0]:
(df_clean
 .write
 .option('header', 'false')
 .option("delimiter", "\t")
 .csv("/mnt/inflation/df_clean.csv"))

In [0]:
location = df1.select("location")

In [0]:
(location
 .write
 .option('header', 'false')
 .option("delimiter", "\t")
 .csv("/mnt/inflation/Location.csv"))

## 4. Feature Enginnering & EDA

- Lets extarct datetime features from created at column For EDA and analysis purpose
- We will focus on feature tarnsfromation while tarining model in pipline 

In [0]:
df_clean.show(5)

+---------------+--------------------+------------+----+--------------------+
|    screen_name|               tweet|    location| geo|          created_at|
+---------------+--------------------+------------+----+--------------------+
|deduped_nytimes|rt breaking news ...|        None|None|Tue Dec 13 13:42:...|
|       GregDaco|inflation coming ...|New York, NY|None|Tue Dec 13 13:43:...|
|  jerseyboy0423|rt thanks to infl...|  Newark, NJ|None|Tue Dec 13 13:45:...|
|   adamzuercher|stock futures are...| Findlay, OH|None|Tue Dec 13 13:45:...|
|     PatRaines7|rt why did not an...|        None|None|Tue Dec 13 13:46:...|
+---------------+--------------------+------------+----+--------------------+
only showing top 5 rows



###### Geo column

In [0]:
null_geo_count = df_clean.filter(col('geo').isNull()).count()
print(f"Number of rows where `geo` is null: {null_geo_count}")

Number of rows where `geo` is null: 1251


In [0]:
unique_geo_values = df_clean.select(col('geo')).distinct()
print(f"Unique `geo` values:\n{unique_geo_values.show(truncate=False)}")

+--------------------------------------------------------------+
|geo                                                           |
+--------------------------------------------------------------+
|None                                                          |
|{'type': 'Point', 'coordinates': [43.7742911, -79.7164011]}   |
|null                                                          |
| Dunfermline Abbey                                            |
|{'type': 'Point', 'coordinates': [38.95218019, -77.05806888]} |
|{'type': 'Point', 'coordinates': [17.42413, 78.45788]}        |
|{'type': 'Point', 'coordinates': [40.42300032, -98.73722448]} |
|{'type': 'Point', 'coordinates': [-33.8637451, 151.2037022]}  |
|{'type': 'Point', 'coordinates': [39.73817686, -104.98921759]}|
|{'type': 'Point', 'coordinates': [23.716667, 44.116667]}      |
|{'type': 'Point', 'coordinates': [34.56530698, -112.39506827]}|
|{'type': 'Point', 'coordinates': [48.2951, -122.659]}         |
|{'type': 'Point', 'coord

In [0]:
# not_null_geo_df = df_clean.filter(col('geo').isNotNull())
unique_geo_values.count()

Out[22]: 23

###### Location column

In [ ]:
# counting No. of tweets for each location
tweet_counts_by_location = df_clean.groupBy("location").count().orderBy("count", ascending=False)
display(tweet_counts_by_location)
tweet_counts_by_location.count()

### Output Example (69263 Rows)

# location	count
# NA	375890
# united states	14556
# 11737
# usa	7706
# london	4744
# london england	4429
# florida usa	4358
# india	4238
# canada	4194

In [0]:
distinct_user_count = df_clean.select("screen_name").distinct().count()
print("Distinct user count:", distinct_user_count)

Distinct user count: 451262


In [0]:
distinct_location = df_clean.select("location").distinct().count()
print("Distinct location:", distinct_location)

Distinct location: 90291


###### screen_name column

In [ ]:
# Counting No. of tweets per user
tweet_counts = df_clean.groupBy("screen_name").agg(count("*").alias("tweet_count"))
most_active_users = tweet_counts.orderBy(desc("tweet_count"))
display(most_active_users)

## 5. Modelling

(LDA)

### spark LDA model

In [0]:
# Splitting the data into training and testing sets
train_data, test_data = df_clean.randomSplit([0.8, 0.2], seed=42)

# Preprocessing steps (Transformatin steps)
regexTokenizer = RegexTokenizer(inputCol="tweet", outputCol="tokens", pattern="\\s+")
stopwordsRemover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
ngram = NGram(n=2, inputCol="filtered_tokens", outputCol="ngrams")
countVectorizer = CountVectorizer(inputCol="ngrams", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Preprocessing pipeline
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram, countVectorizer, idf])

# Fitting the pipeline - training data
model = pipeline.fit(train_data)

# Transforming the training and testing data using the fitted pipeline
train_transformed = model.transform(train_data)
test_transformed = model.transform(test_data)

# LDA model (Spark inbuilt)
lda = LDA(k=20, maxIter=10, featuresCol="features")

# Fitting the LDA model to the transformed training data
lda_model = lda.fit(train_transformed)

##### Predicting on test dataset 

In [0]:
# topic distribuions for Test data
predictions = lda_model.transform(test_transformed)

# # Interpreting the results
# for row in predictions.select("topicDistribution", "tweet").collect():
#     print("Predicted topic distribution: ", row.topicDistribution)
#     print("Document text: ", row.tweet)

In [0]:
## Lets check topics for test data
display(predictions.take(5))

screen_name,tweet,location,geo,created_at,tokens,filtered_tokens,ngrams,rawFeatures,features,topicDistribution
007DayTraders,rt sadly cpi is complete trash and does not include proper weighting of the most expensive items such as food and,None,None,Tue Dec 13 18:56:03 +0000 2022,"List(rt, sadly, cpi, is, complete, trash, and, does, not, include, proper, weighting, of, the, most, expensive, items, such, as, food, and)","List(rt, sadly, cpi, complete, trash, include, proper, weighting, expensive, items, food)","List(rt sadly, sadly cpi, cpi complete, complete trash, trash include, include proper, proper weighting, weighting expensive, expensive items, items food)","Map(vectorType -> sparse, length -> 262144, indices -> List(18737, 24994, 27660, 27895, 27960, 28146, 28295, 28341, 28570, 28680), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(18737, 24994, 27660, 27895, 27960, 28146, 28295, 28341, 28570, 28680), values -> List(9.864648388205458, 10.179729434845353, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.340072084920532))","Map(vectorType -> dense, length -> 20, values -> List(0.9911898744256012, 4.6293701616150385E-4, 4.62852553081183E-4, 4.6475889609018264E-4, 4.6382031561182426E-4, 4.63607569814898E-4, 4.636885515196945E-4, 4.6278834192582994E-4, 4.642121244785682E-4, 4.6317018544652787E-4, 4.6425425573610866E-4, 4.637255438009523E-4, 4.6398464852113975E-4, 4.6337957749212593E-4, 4.6657893600184926E-4, 4.633566944149604E-4, 4.638930521120803E-4, 4.628234265209655E-4, 4.632783822485352E-4, 4.6301550342010524E-4))"
007dmax,rt f me lost count of the amount of grade a daily mail utter bullshite in this one tweet,None,None,Wed Dec 14 10:57:07 +0000 2022,"List(rt, f, me, lost, count, of, the, amount, of, grade, a, daily, mail, utter, bullshite, in, this, one, tweet)","List(rt, f, lost, count, amount, grade, daily, mail, utter, bullshite, one, tweet)","List(rt f, f lost, lost count, count amount, amount grade, grade daily, daily mail, mail utter, utter bullshite, bullshite one, one tweet)","Map(vectorType -> sparse, length -> 262144, indices -> List(19278, 38889, 55931, 194519), values -> List(1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(19278, 38889, 55931, 194519), values -> List(9.892047362393573, 10.642352956793466, 10.990659651061682, 12.376954012181573))","Map(vectorType -> dense, length -> 20, values -> List(0.29906721907297384, 0.001067286846294804, 0.0010670921194399481, 0.0010714871336325207, 0.0010693232652408864, 0.0010688327864749425, 0.0010690194882284141, 0.0010669440813088355, 0.0010702265680594652, 0.0010678244104994226, 0.0010703237003627441, 0.0010691047720438167, 0.3323466600449304, 0.0010683071585255873, 0.0010756831779374716, 0.0010682544009060087, 0.0010694909582985024, 0.001067024969122433, 0.3504124272480715, 0.001067467797648502))"
00DMadHatter00,they will launder the people s money though,Here & There,None,Sat Dec 10 14:40:29 +0000 2022,"List(they, will, launder, the, people, s, money, though)","List(launder, people, money, though)","List(launder people, people money, money though)","Map(vectorType -> sparse, length -> 262144, indices -> List(5591, 38682, 132734), values -> List(1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(5591, 38682, 132734), values -> List(8.6080318503941, 10.642352956793466, 11.866128388415582))","Map(vectorType -> dense, length -> 20, values -> List(0.0014990825039314297, 0.3570307076197461, 0.0014924379727774904, 0.001498584858121939, 0.0014955584817734006, 0.0014948724754231168, 0.0014951335958014993, 0.0014922309307883932, 0.0014968218228790207, 0.0014934621582310729, 0.6160522005318234, 0.001495252875066288, 0.0014960883414314504, 0.001494137340306339, 0.0015044534656529632, 0.0014940635440979636, 0.0014957929959128829, 0.00149234405

In [0]:
# Getting the vocabulary from the CVstep
vocabArray = model.stages[-2].vocabulary

# conerting the topic ditribution to a list of topic terms
def get_topic_terms(topicDistribution):
    topic_terms = [vocabArray[i] for i in topicDistribution.argsort()[::-1][::]]
    return topic_terms

get_topic_terms_udf = udf(get_topic_terms, ArrayType(StringType()))

# Applying the UDF to the topicDistribution column in the predictions df
predictions_with_topic_terms = predictions.withColumn("topicTerms", get_topic_terms_udf("topicDistribution"))
display(predictions_with_topic_terms.take(5))

screen_name,tweet,location,geo,created_at,tokens,filtered_tokens,ngrams,rawFeatures,features,topicDistribution,topicTerms
007DayTraders,rt sadly cpi is complete trash and does not include proper weighting of the most expensive items such as food and,None,None,Tue Dec 13 18:56:03 +0000 2022,"List(rt, sadly, cpi, is, complete, trash, and, does, not, include, proper, weighting, of, the, most, expensive, items, such, as, food, and)","List(rt, sadly, cpi, complete, trash, include, proper, weighting, expensive, items, food)","List(rt sadly, sadly cpi, cpi complete, complete trash, trash include, include proper, proper weighting, weighting expensive, expensive items, items food)","Map(vectorType -> sparse, length -> 262144, indices -> List(18737, 24994, 27660, 27895, 27960, 28146, 28295, 28341, 28570, 28680), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(18737, 24994, 27660, 27895, 27960, 28146, 28295, 28341, 28570, 28680), values -> List(9.864648388205458, 10.179729434845353, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.297512470501736, 10.340072084920532))","Map(vectorType -> dense, length -> 20, values -> List(0.9911898744256012, 4.6293701616150385E-4, 4.62852553081183E-4, 4.6475889609018264E-4, 4.6382031561182426E-4, 4.63607569814898E-4, 4.636885515196945E-4, 4.6278834192582994E-4, 4.642121244785682E-4, 4.6317018544652787E-4, 4.6425425573610866E-4, 4.637255438009523E-4, 4.6398464852113975E-4, 4.6337957749212593E-4, 4.6657893600184926E-4, 4.633566944149604E-4, 4.638930521120803E-4, 4.628234265209655E-4, 4.632783822485352E-4, 4.6301550342010524E-4))","List(rt inflation, false told, gas prices, announcements last, couple weeks, easing signed, inflation reduction, rt major, weeks inflations, signed respect, major announcements, respect marriag, rt breaking, pay rise, inflations easing, interest rates, corporate profits, told us, reduction act, last couple)"
007dmax,rt f me lost count of the amount of grade a daily mail utter bullshite in this one tweet,None,None,Wed Dec 14 10:57:07 +0000 2022,"List(rt, f, me, lost, count, of, the, amount, of, grade, a, daily, mail, utter, bullshite, in, this, one, tweet)","List(rt, f, lost, count, amount, grade, daily, mail, utter, bullshite, one, tweet)","List(rt f, f lost, lost count, count amount, amount grade, grade daily, daily mail, mail utter, utter bullshite, bullshite one, one tweet)","Map(vectorType -> sparse, length -> 262144, indices -> List(19278, 38889, 55931, 194519), values -> List(1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(19278, 38889, 55931, 194519), values -> List(9.892047362393573, 10.642352956793466, 10.990659651061682, 12.376954012181573))","Map(vectorType -> dense, length -> 20, values -> List(0.29906721907297384, 0.001067286846294804, 0.0010670921194399481, 0.0010714871336325207, 0.0010693232652408864, 0.0010688327864749425, 0.0010690194882284141, 0.0010669440813088355, 0.0010702265680594652, 0.0010678244104994226, 0.0010703237003627441, 0.0010691047720438167, 0.3323466600449304, 0.0010683071585255873, 0.0010756831779374716, 0.0010682544009060087, 0.0010694909582985024, 0.001067024969122433, 0.3504124272480715, 0.001067467797648502))","List(pay rise, easing signed, rt inflation, false told, gas prices, announcements last, couple weeks, inflation reduction, rt major, weeks inflations, signed respect, major announcements, respect marriag, rt breaking, inflations easing, interest rates, corporate profits, told us, reduction act, last couple)"
00DMadHatter00,they will launder the people s money though,Here & There,None,Sat Dec 10 14:40:29 +0000 2022,"List(they, will, launder, the, people, s, money, though)","List(launder, people, money, though)","List(launder people, people money, money though)","Map(vectorType -> sparse, length -> 262144, indices -> List(5591, 38682, 132734), values

##### Topic distribution (topic probability score) with topic terms 

In [0]:
display(predictions_with_topic_terms.take(5))

screen_name,tweet,location,geo,created_at,tokens,filtered_tokens,ngrams,rawFeatures,features,topicDistribution,topicTerms
00wasp001,so if the boats stopped completely tomorrow do you thi,UK,None,Wed Dec 14 17:59:49 +0000 2022,"List(so, if, the, boats, stopped, completely, tomorrow, do, you, thi)","List(boats, stopped, completely, tomorrow, thi)","List(boats stopped, stopped completely, completely tomorrow, tomorrow thi)","Map(vectorType -> sparse, length -> 262144, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 262144, indices -> List(), values -> List())","Map(vectorType -> dense, length -> 20, values -> List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0))","List(interest rates, pay rise, told us, corporate profits, gas prices, rt major, major announcements, signed respect, last couple, couple weeks, inflations easing, announcements last, weeks inflations, easing signed, respect marriag, false told, rt breaking, inflation reduction, reduction act, rt inflation)"
07Cat31,rt republicans will return the committee to its proper role rooting out waste fraud abuse and mismanagement in the federal g,"Nevada, USA",None,Thu Dec 15 01:58:41 +0000 2022,"List(rt, republicans, will, return, the, committee, to, its, proper, role, rooting, out, waste, fraud, abuse, and, mismanagement, in, the, federal, g)","List(rt, republicans, return, committee, proper, role, rooting, waste, fraud, abuse, mismanagement, federal, g)","List(rt republicans, republicans return, return committee, committee proper, proper role, role rooting, rooting waste, waste fraud, fraud abuse, abuse mismanagement, mismanagement federal, federal g)","Map(vectorType -> sparse, length -> 262144, indices -> List(259, 23795, 23856, 23927, 24472, 24484, 25231, 30291, 31408, 31682, 31838, 34111), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(259, 23795, 23856, 23927, 24472, 24484, 25231, 30291, 31408, 31682, 31838, 34111), values -> List(5.711974452177938, 10.142791251021253, 10.142791251021253, 10.142791251021253, 10.142791251021253, 10.142791251021253, 10.179158895192128, 10.38395330783814, 10.430473323473032, 10.430473323473032, 10.430473323473032, 10.479263487642465))","Map(vectorType -> dense, length -> 20, values -> List(4.000987650260926E-4, 4.001611651350039E-4, 4.0080293589628453E-4, 4.0060428940556894E-4, 4.0101664131722385E-4, 4.007392091361693E-4, 4.0029755485529185E-4, 4.001253925874679E-4, 4.0059830608138523E-4, 4.004339164744415E-4, 4.0053472099176626E-4, 4.008808239350384E-4, 4.0103456867079815E-4, 3.995129668474556E-4, 4.027686659101397E-4, 0.9923863726478098, 4.0066822623486745E-4, 4.0069665835050744E-4, 4.0182715147996805E-4, 4.00825393854719E-4))","List(rt breaking, false told, pay rise, easing signed, rt major, weeks inflations, interest rates, corporate profits, major announcements, reduction act, inflation reduction, gas prices, couple weeks, announcements last, inflations easing, signed respect, told us, last couple, rt inflation, respect marriag)"
0o0ofaureo0o0,rt nasdaq futures jump after us inflation data show prices grew less than forecast last month,None,None,Tue Dec 13 15:11:45 +0000 2022,"List(rt, nasdaq, futures, jump, after, us, inflation, data, show, prices, grew, less, than, forecast, last, month)","List(rt, nasdaq, futures, jump, us, inflation, data, show, prices, grew, less, forecast, last, month)","List(rt nasdaq, nasdaq futures, futures jump, jump us, us inflation, inflation data, data show, show prices, prices grew, grew less, less forecast, forecast last, last month)","Map(vectorType -> sparse, length -> 262144, indices -> List(22, 70, 80, 6571, 14677, 19456, 29434, 33272, 40767, 50574, 58271, 64596, 66543), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(22, 70, 80, 6571, 146

##### Coherence Score 

tried using the logLikelihood below for cohernce score but takes too much computing power and time

In [0]:
# Coherence score
log_likelihood = lda_model.logLikelihood(train_transformed)
# num_docs = train_transformed.count()
# coherence_score = 2.0 * log_likelihood / (num_docs * (num_docs - 1))
# print("Coherence score: {}".format(coherence_score))

##### Lets check list of topic term of our tweets 

In [0]:
# Lrts get the terms for each topic
topics = lda_model.describeTopics()
vocabArray = model.stages[-2].vocabulary

topics_rdd = topics.rdd.map(lambda row: row['termIndices']).map(lambda idx_list: [vocabArray[idx] for idx in idx_list])
for idx, topic in enumerate(topics_rdd.collect()):
    print("Topic {0}: {1}\n".format(idx, topic))


Topic 0: ['control inflation', 'inflation must', 'working people', 'rt say', 'say control', 'take aim', 'must take', 'profits working', 'aim corporate', 'gas prices']

Topic 1: ['victory lap', 'joe biden', 'lap inflation', 'take victory', 'rt man', 'biden take', 'incompetent corrupt', 'man incompetent', 'corrupt joe', 'inflation collapsing']

Topic 2: ['gas prices', 'inflation declining', 'rt well', 'straight months', 'prices common', 'declining straight', 'common price', 'price unemployment', 'near ye', 'inflation decreasing']

Topic 3: ['inflation easing', 'signs inflation', 'job creation', 'near year', 'early signs', 'year low', 'low gas', 'historic job', 'rt historic', 'easing unemployment']

Topic 4: ['retail inflation', 'inflation declines', 'declines month', 'month low', 'low november', 'rt retail', 'let us', 'printing money', 'modi govt', 'november measures']

Topic 5: ['reduction act', 'inflation reduction', 'rt watch', 'tax cut', 'cut inflation', 'another tax', 'america getti

##### Evaluation metric

It's hard to get coherence score while using LDA model from spark 
however, we car try training out model with Gensim LDA model Which is more easy to interpret as well as easy to transfrom and acess the corpus of the topics

#### Saving the prediction in S3 bucket 

In [0]:
predictions = predictions_with_topic_terms

In [0]:
(predictions
 .write
 .parquet('/mnt/inflation/Sparkpred_lda.parquet'))

### Analyzing the Model
##### Possible interpration from term of topics 
- topic 0 - Inflation concerns India/ politics and government policies.
- topic 1 - Inflation control strategy.
- topic 2 - Corporate profitability analysis.
- topic 4 - Inflation trending downwards.
- topic 8 - Russian oil purchases.

#### Gensim LDA model

##### Spark - preprocessing steps and train model on Gensim Lda model

In [0]:
# Splitting data into trainng and testing set
train_data, test_data = df_clean.randomSplit([0.8, 0.2], seed=42)

# preprocessing steps
regexTokenizer = RegexTokenizer(inputCol="tweet", outputCol="tokens", pattern="\\s+")
stopwordsRemover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
ngram = NGram(n=2, inputCol="filtered_tokens", outputCol="ngrams")
countVectorizer = CountVectorizer(inputCol="ngrams", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

# pipeline for preprocessing steps
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, ngram, countVectorizer, idf])

# fitting pipeline to the training data
model = pipeline.fit(train_data)

# Transforming the training and testing data using the fitted pipeline
train_transformed = model.transform(train_data)
test_transformed = model.transform(test_data)


texts = train_transformed.select("ngrams").rdd.flatMap(lambda x: x).collect()
texts = [text.split() if isinstance(text, str) else text for text in texts]

# Creating Gensim dictionary
dictionary = gensim.corpora.Dictionary(texts)

#creating corpus
corpus = []
for text in texts:
    bow = dictionary.doc2bow(text)
    corpus.append(bow)

# Training LDA model using Gensim
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, iterations=10)


##### Coherence score and LDA perplexity for model comparison

In [0]:
# Coherence score for eavulating the model
coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print('coherence_score: ', coherence_score)
# Perplexity for checking differnec between 2 model 
print('LDA Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

coherence_score:  0.33436422530193927
LDA Perplexity:  -23.07713527142825


##### Term topic and their weights

In [0]:
topics = lda_model.print_topics()
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")

Topic 0: 0.027*"inflation pay" + 0.016*"rt pay" + 0.016*"next year" + 0.015*"fuelled inflation" + 0.015*"pay fuelled" + 0.015*"years profit" + 0.015*"squeezed years" + 0.015*"pay squeezed" + 0.015*"profit pushed" + 0.014*"rt president"
Topic 1: 0.024*"rt inflation" + 0.022*"pay rises" + 0.021*"inflation lower" + 0.020*"public sector" + 0.015*"climate change" + 0.014*"gas prices" + 0.014*"gdp jobs" + 0.013*"prices gdp" + 0.013*"lower gas" + 0.012*"jobs manufacturing"
Topic 2: 0.026*"inflation still" + 0.015*"inflation inflation" + 0.014*"annual inflation" + 0.014*"still high" + 0.013*"food price" + 0.011*"public sector" + 0.011*"people said" + 0.010*"inflation expectations" + 0.010*"rt wrote" + 0.010*"far enough"
Topic 3: 0.095*"told us" + 0.062*"false told" + 0.033*"rt told" + 0.033*"zero inflation" + 0.031*"russian collusion" + 0.031*"inflation false" + 0.031*"us russian" + 0.031*"us zero" + 0.031*"collusion false" + 0.031*"us twitter"
Topic 4: 0.041*"us inflation" + 0.028*"last quart

##### How to interpret the reuslt 
for eg.,
Topic 0:- These words suggest that the topic is about reporting on the latest inflation numbers and how they are affecting corporate profits, with a focus on record highs.
Topic 4:- interest rates, federal reserve, rate hikes

##### juts terms of topic

In [0]:
topics = lda_model.show_topics(num_topics=-1, num_words=10, formatted=False)
for topic in topics:
    print(f"Topic {topic[0]}: {[word[0] for word in topic[1]]}")

Topic 0: ['inflation pay', 'rt pay', 'next year', 'fuelled inflation', 'pay fuelled', 'years profit', 'squeezed years', 'pay squeezed', 'profit pushed', 'rt president']
Topic 1: ['rt inflation', 'pay rises', 'inflation lower', 'public sector', 'climate change', 'gas prices', 'gdp jobs', 'prices gdp', 'lower gas', 'jobs manufacturing']
Topic 2: ['inflation still', 'inflation inflation', 'annual inflation', 'still high', 'food price', 'public sector', 'people said', 'inflation expectations', 'rt wrote', 'far enough']
Topic 3: ['told us', 'false told', 'rt told', 'zero inflation', 'russian collusion', 'inflation false', 'us russian', 'us zero', 'collusion false', 'us twitter']
Topic 4: ['us inflation', 'last quarter', 'stock buybacks', 'billion stock', 'raked billion', 'announced billion', 'billion profits', 'profits last', 'quarter announced', 'exxonmobil raked']
Topic 5: ['major announcements', 'rt major', 'signed respect', 'last couple', 'couple weeks', 'inflations easing', 'announceme

##### visual representation of the topics

- The bubbles represent the prevalence of topics and their similarity is based on word overlap.
- (More strong and distinct topic from documnet if the bubble is not overlapping)
- (Moving the silder of relevence metric to left help us to remove/ filter most comman words which is better it helps to avoid common words which were not removed by stopwrod transformation)

In [0]:
# using pyLDAvis
# It's not visible on this GitHub page, but I've included an image to provide a clearer depiction of its appearance.
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
html = pyLDAvis.prepared_data_to_html(vis)
displayHTML(html)

##### Test data 

In [0]:
# Evaluating the LDA model on the transformed testing data
test_texts = test_transformed.select("filtered_tokens").rdd.flatMap(lambda x: x).collect()
test_texts = [test_texts.split() if isinstance(text, str) else text for text in test_texts]

# Creating Gensim dictionary
test_dictionary = gensim.corpora.Dictionary(test_texts)

corpus = []
for text in test_texts:
    bow = test_dictionary.doc2bow(text)
    corpus.append(bow)

# Train an LDA model using Gensim
lda_model_test = LdaModel(corpus=corpus, id2word=test_dictionary, num_topics=20, iterations=10)

coherence_model_test= CoherenceModel(model=lda_model_test, texts=test_texts, dictionary=test_dictionary, coherence='c_v')
coherence_score_test = coherence_model_test.get_coherence()

print("Test_Coherence_Score: ", coherence_score_test)

# Perplexity for checking differnec between 2 model 
print('Test_LDA_Perplexity: ', lda_model_test.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

Test_Coherence_Score:  0.30149497846479506
Test_LDA_Perplexity:  -9.39952669029089


#### Gensim preprocessing and model (just to compare result from spark LDA and Gensim LDA model)

In [0]:
# Preprocessing functions
def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)

def remove_stopwords(texts):
    stop_words = set(stopwords.words('english'))
    stop_words.update(['from', 'subject', 're', 'edu', 'use', 'rt'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram_mod[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts):
    lemmatizer = WordNetLemmatizer()
    texts_out = []
    for sent in texts:
        doc = [lemmatizer.lemmatize(token) for token in sent]
        texts_out.append(doc)
    return texts_out

# Spliting the data into train and test sets
train_data, test_data = df_clean.randomSplit([0.8, 0.2], seed=42)

# Applying the preprocessing steps only on the train set
train_tweets = train_data.select("tweet").rdd.flatMap(list).collect()
train_words = list(sent_to_words(train_tweets))
train_words_nostops = remove_stopwords(train_words)
train_words_bigrams = make_bigrams(train_words_nostops)
train_words_trigrams = make_trigrams(train_words_nostops)
train_data_lemmatized = lemmatization(train_words_trigrams)

# Printting the preprocessed train data
print(train_data_lemmatized[:1])

[['lost', 'count', 'amount', 'grade', 'daily_mail', 'utter', 'bullshite', 'one', 'tweet']]


In [0]:
# Creating Dictionary
id2word = corpora.Dictionary(train_data_lemmatized)
# Creating Corpus
texts = train_data_lemmatized
# TDF
corpus = [id2word.doc2bow(text) for text in texts]

In [0]:
id2word[0]

Out[28]: 'amount'

In [0]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Out[29]: [[('amount', 1),
  ('bullshite', 1),
  ('count', 1),
  ('daily_mail', 1),
  ('grade', 1),
  ('lost', 1),
  ('one', 1),
  ('tweet', 1),
  ('utter', 1)]]

In [0]:
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=10, iterations=10)

In [0]:
# Keywords in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.091*"price" + 0.079*"inflation" + 0.075*"gas" + 0.038*"republican" + 0.025*"still" + 0.024*"biden" + 0.024*"unemployment" + 0.022*"plan" + 0.020*"billion" + 0.016*"corporate"'), (1, '0.069*"inflation" + 0.035*"increase" + 0.027*"country" + 0.020*"price" + 0.017*"food" + 0.014*"increased" + 0.014*"economy" + 0.013*"month" + 0.013*"government" + 0.013*"unemployment"'), (2, '0.099*"inflation" + 0.086*"last" + 0.065*"week" + 0.061*"major" + 0.056*"easing" + 0.052*"respect" + 0.052*"announcement" + 0.051*"signed" + 0.050*"couple" + 0.049*"marriag"'), (3, '0.107*"inflation" + 0.038*"high" + 0.037*"profit" + 0.034*"act" + 0.031*"record" + 0.031*"reduction" + 0.022*"tax" + 0.022*"hit" + 0.020*"latest" + 0.016*"america"'), (4, '0.065*"inflation" + 0.021*"money" + 0.014*"amp" + 0.013*"new" + 0.012*"way" + 0.012*"spending" + 0.011*"know" + 0.010*"fight" + 0.010*"like" + 0.009*"mean"'), (5, '0.139*"inflation" + 0.034*"november" + 0.021*"year" + 0.020*"price" + 0.020*"month" + 0.018*"lower"

In [0]:
# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=train_data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Coherence Score:  0.2640968403781919

Perplexity:  -7.671148248353711


In [0]:
# Visualing the topics
# It's not visible on this GitHub page, but I've included an image to provide a clearer depiction of its appearance.
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

# 6. Conclusion

In this project, we applied a LDA model and Gensim LDA (Latent Dirichlet Allocation) model to a collection of Twitter tweets discussing inflation in the aftermath of the COVID-19 pandemic. Through this analysis, we uncovered ten distinct topics that capture various facets of the inflation-related discussions on the platform.

These topics range from economic factors such as price, government policies, and corporate profits to discussions on the Federal Reserve's role, public sentiment about wage increases, and even unrelated themes that emerged in the Twitter discourse.

The insights gained from this analysis can be valuable for businesses and policymakers. By monitoring these topics and understanding their evolution over time, businesses can adapt their strategies to respond to changing economic conditions, rising costs, and shifting public sentiment. This analysis also serves as a reminder of the interconnectedness of economic factors and the importance of staying informed and engaged with the community.

In conclusion, this project illustrates the power of natural language processing and topic modeling techniques in extracting meaningful insights from social media data. It encourages businesses and decision-makers to address inflation-related challenges and opportunities in a dynamic post-COVID-19 economic landscape.
